In [1]:
import os
os.chdir("../")
# os.chdir("src/")
%pwd

'c:\\Users\\Nilesh\\Documents\\GitHub\\End-to-End-Diabetes-Detection'

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str    
    penalty: str
    C: float
    solver: str
    max_iter: int
    class_weight: dict
    multi_class: str    
    target_column: str

  

In [33]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            penalty = params.penalty,
            C = params.C,
            solver = params.solver,
            max_iter = params.max_iter,
            class_weight = params.class_weight,
            multi_class = params.multi_class,
            target_column = schema.name
            
        )

        return model_trainer_config

In [35]:
import pandas as pd
import os
from src.mlProject import logger
from sklearn.linear_model import LogisticRegression
import joblib

In [36]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Convert target variables to 1-dimensional arrays
        train_y = train_y.values.ravel()
        test_y = test_y.values.ravel()

        lr = LogisticRegression(penalty=self.config.penalty,
                                C=self.config.C,
                                solver=self.config.solver,
                                max_iter=self.config.max_iter,
                                class_weight=self.config.class_weight,
                                multi_class=self.config.multi_class)

        lr.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [37]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-21 20:21:12,808: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-21 20:21:12,808: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-21 20:21:12,808: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-21 20:21:12,808: INFO: common: created directory at: artifacts]
[2024-02-21 20:21:12,823: INFO: common: created directory at: artifacts/model_trainer]
